# Variational autoencoder

The variational autoencoder should be able to identify abnormal heartbeat patterns.

In [3]:
from mads_datasets.base import BaseDatastreamer
from mltrainer.preprocessors import BasePreprocessor
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from torch import nn
import torch
import gin
from streamer import VAEstreamer
import vae
from loguru import logger

import sys
import datasets, metrics
import mltrainer
from mltrainer import ReportTypes, Trainer, TrainerSettings


In [11]:
   
# use the binary data for training the Variational Autoencoder
trainfileVAE = Path('../data/heart_train.parq').resolve()
testfileVAE = Path('../data/heart_test.parq').resolve()

# Remove outliers for training the VAE
traindatasetVAE = datasets.HeartDataset1D(trainfileVAE, target="target", outliersRemoval=True)
testdatasetVAE = datasets.HeartDataset1D(testfileVAE, target="target", outliersRemoval=True)

validationSetVAE = datasets.HeartDataset1D(testfileVAE, target="target", outliersRemoval=False)

trainstreamerVAE = VAEstreamer(traindatasetVAE, batchsize=32).stream()
teststreamerVAE = VAEstreamer(testdatasetVAE, batchsize=32).stream()
validationstreamerVAE = VAEstreamer(validationSetVAE, batchsize=32).stream()


In [13]:
gin.parse_config_file(Path('../src/config.gin').resolve())

X1, X2 = next(trainstreamerVAE)

encoder = vae.Encoder()
decoder = vae.Decoder()

latent = encoder(X1)
logger.info(f"the latent shape : {latent.shape}")

x = decoder(latent)
logger.info(f"the shape after: {x.shape}")

lossfn = vae.ReconstructionLoss()
loss = lossfn(x, X2)
logger.info(f"Untrained loss: {loss}")

logger.info(f"starting training for {100} epochs")
autoencoder = vae.AutoEncoder()

settings = TrainerSettings(
    epochs=100,
    metrics=[lossfn],
    logdir="logs",
    train_steps=200,
    valid_steps=200,
    reporttypes=[ReportTypes.TENSORBOARD],
    scheduler_kwargs={"factor": 0.5, "patience": 10},
)

trainer = Trainer(
    model=autoencoder,
    settings=settings,
    loss_fn=lossfn,
    optimizer=torch.optim.Adam,
    traindataloader=trainstreamerVAE,
    validdataloader=teststreamerVAE,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau,
)
trainer.loop()

2024-06-20 11:37:28.821 | INFO     | __main__:<module>:9 - the latent shape : torch.Size([32, 2])
2024-06-20 11:37:28.826 | INFO     | __main__:<module>:12 - the shape after: torch.Size([32, 192])
2024-06-20 11:37:28.830 | INFO     | __main__:<module>:16 - Untrained loss: 11.769255638122559
2024-06-20 11:37:28.831 | INFO     | __main__:<module>:18 - starting training for 100 epochs
2024-06-20 11:37:28.836 | INFO     | mltrainer.settings:check_path:61 - Created logdir /home/azureuser/code/nickyvanoorschot_mads_exam_24/notebooks/logs
2024-06-20 11:37:28.837 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to logs/20240620-113728
2024-06-20 11:37:29.271 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 200/200 [00:01<00:00, 165.08it/s]
2024-06-20 11:37:30.782 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 3.4280 test 2.3985 metric ['2.3985']
100%|██████████| 200/200 [00:01<00

In [ ]:
import seaborn as sns
import numpy as np

y_true = []
y_pred = []

testdata = validationstreamerVAE.stream()
for _ in range(len(validationstreamerVAE)):
    X, y = next(testdata)
    yhat = autoencoder(X)
    yhat = yhat.argmax(dim=1) # we get the one with the highest probability
    y_pred.append(yhat.cpu().tolist())
    y_true.append(y.cpu().tolist())

yhat = [x for y in y_pred for x in y]
y = [x for y in y_true for x in y]

cfm = confusion_matrix(y, yhat)
# cfm = cfm / np.sum(cfm, axis=1, keepdims=True)

plot = sns.heatmap(cfm, annot=cfm, fmt=".3f")
plot.set(xlabel="Predicted", ylabel="Target")